In [1]:
import numpy as np
import pickle
import json
import os
import sys
import copy
import sklearn.preprocessing
import models
from sklearn.neighbors import NearestNeighbors
import utils

In [2]:
data='fb15k'
DATA_DIR = "../../data/"+data
DUMP_FILE = "../dumps/"+data+"_distmult_dump_norm.pkl"
MODEL_TYPE = data
# mining_dir=data+"_low_thresh"
mining_dir=data+"_rule_mining_tmp"
os.system("mkdir -p "+mining_dir)

0

In [3]:
train_data = utils.read_data(os.path.join(DATA_DIR,"train.txt"))
# dev_data = read_data(os.path.join(DATA_DIR,"valid.txt"))
# test_data = read_data(os.path.join(DATA_DIR,"test.txt"))

In [4]:
dump=utils.load_pickle(DUMP_FILE)
dump.keys()

dict_keys(['tail_rel_type', 'entity_type', 'entity_to_id', 'head_rel_type', 'entity_real', 'rel_real', 'relation_to_id'])

In [5]:
model=models.TypedDM(DUMP_FILE)

In [6]:
mapped_train_data = utils.map_data(train_data,dump)
# mapped_dev_data = map_data(dev_data)
# mapped_test_data = map_data(test_data)

In [7]:
entity_to_rel=utils.get_ent_to_rel(mapped_train_data)

In [8]:
rules_1_path=os.path.join(mining_dir,"1_sup=1_conf=0.pkl")
rules_2_path=os.path.join(mining_dir,"2_sup=1_conf=0.pkl")
rules_3_path=os.path.join(mining_dir,"3_sup=4_conf=0.pkl")
rules_1=utils.load_pickle(rules_1_path)
rules_2=utils.load_pickle(rules_2_path)
rules_3=utils.load_pickle(rules_3_path)
print(len(rules_1),len(rules_2),len(rules_3))

4296 9328 102926374


In [9]:
# print(rules_1[0])
# print(rules_2[0])
# print(rules_3[0])

In [10]:
def add_relation_body(rules,relation_to_body):
    for rule in rules:
        if rule[1] not in relation_to_body:
            relation_to_body[rule[1]]=[]
        relation_to_body[rule[1]].append(rule[0])
    return relation_to_body

In [11]:
# relation_to_body={}
# relation_to_body=add_relation_body(rules_1,relation_to_body)
# relation_to_body=add_relation_body(rules_2,relation_to_body)
# relation_to_body=add_relation_body(rules_3,relation_to_body)
# print(len(relation_to_body))


In [25]:
# print(rules_1[4])
# print(rules_2[4])
print(rules_3[-100:])

[((1230, 43, 91), 914, 3590, 1), ((1291, 43, 91), 764, 3590, 1), ((1291, 43, 91), 864, 3590, 1), ((1291, 43, 91), 217, 3590, 1), ((1291, 43, 91), 23, 3590, 1), ((1291, 43, 91), 446, 3590, 1), ((1291, 43, 91), 648, 3590, 1), ((1291, 43, 91), 236, 3590, 1), ((1291, 43, 91), 943, 3590, 1), ((1291, 43, 91), 1291, 3590, 1), ((1291, 43, 91), 114, 3590, 1), ((1291, 43, 91), 1291, 3590, 1), ((1291, 43, 91), 28, 3590, 1), ((1291, 43, 91), 303, 3590, 1), ((1291, 43, 91), 943, 3590, 1), ((1291, 43, 91), 1291, 3590, 1), ((402, 1087, 91), 402, 3590, 1), ((1292, 43, 91), 23, 3590, 1), ((1292, 43, 91), 691, 3590, 1), ((1292, 43, 91), 1205, 3590, 1), ((47, 1087, 91), 490, 3590, 1), ((47, 1087, 91), 490, 3590, 1), ((1297, 43, 91), 41, 3590, 1), ((1297, 43, 91), 154, 3590, 1), ((1297, 43, 91), 149, 3590, 1), ((1297, 43, 91), 305, 3590, 1), ((1297, 43, 91), 1297, 3590, 1), ((1298, 43, 91), 294, 3590, 1), ((1298, 43, 91), 154, 3590, 1), ((1299, 43, 91), 294, 3590, 1), ((1299, 43, 91), 154, 3590, 1), ((110

In [ ]:
_ = [rules.sort(key=lambda x:(x[3]*1.0)/x[2],reverse=True) for rules in [rules_1, rules_2, rules_3]]
rules=rules_1+rules_2+rules_3
print(len(rules))


102939998


In [ ]:
relation_to_body={}
relation_to_body=add_relation_body(rules,relation_to_body)
print(len(relation_to_body))

1281


# Length 1 Rules

In [ ]:
dict_1=utils.get_r_e1e2_dict(mapped_train_data)

# Length 2 Rules

In [ ]:
index_head=utils.get_head_index(mapped_train_data)
dict_2=utils.get_r1r2_e1e2_dict(mapped_train_data,index_head)

## Length 3 Rules

In [ ]:
## Gives entity in path for given relation and body

In [ ]:
re2_e1=utils.get_re2_e1_dict(mapped_train_data)
e1r_e2=utils.get_e1r_e2_dict(mapped_train_data)


In [ ]:
set_mapped_train_data=utils.get_set_mapped_train_data(mapped_train_data)

In [ ]:
def solve_3(fact,body,e1r_e2,re2_e1,set_mapped_train_data):
    r1=body[0]
    r2=body[1]
    r3=body[2]
    e1=fact[0]
    e4=fact[2]
    
    key1=(e1,r1)
    key2=(r3,e4)
    
    if key1 not in e1r_e2:
        return ("",-1)
    if key2 not in re2_e1:
        return ("",-1)
    list1=e1r_e2[key1]
    list2=re2_e1[key2]
    
    for e2 in list1:
        for e3 in list2:
            if (e2,r2,e3) in set_mapped_train_data:
                return (body,(e2,e3))
    
    return ("",-1)
    

In [ ]:
def get_explanation(fact,relation_to_body,dict_1,dict_2,e1r_e2,re2_e1,set_mapped_train_data):
    pair=(fact[0],fact[2])
    rel=fact[1]
    
    r1=0
    r2=0
    r3=0
#     print(r1,r2,r3)
    if rel not in relation_to_body:
        return ("",-1)
#     print("Exists")
    bodies=relation_to_body[rel]
    for body in bodies:
        if isinstance(body,int):
            r1+=1
            if pair not in dict_1[body]:
                continue
            return (body,-1)
        else:
            if len(body)==2:
                r2+=1
                if pair not in dict_2[body]:
                    continue
                return (body,dict_2[body][pair])
            else:
                if len(body)==3:
                    r3+=1
                    temp=solve_3(fact,body,e1r_e2,re2_e1,set_mapped_train_data)
                    if(temp[0]!=""):
                        return temp
    
    return ("",-1)

In [ ]:
# get_explanation((453,37,82),relation_to_body,dict_1,dict_2)

In [ ]:
#triplet_file="/home/cse/btech/cs1150210/scratch/BTP/Interpretable-KBC/logs/fb15k/turk_test_hits10_not_hits1/small_id.txt"
# triplet_file="/home/yatin/hpcscratch/Aman_BTP/Interpretable-KBC-tlp/data/fb15k/test/test_hits_1_ordered_x.txt"
triplet_file="/home/yatin/hpcscratch_copy/texkbc/Interpretable-KBC-tlp/data/fb15k/test/test_hits1_x.txt"
data=utils.read_data(triplet_file)
np_arr=np.array(utils.map_data(data,dump=dump)).astype(np.int32)
# np_arr=np.loadtxt(triplet_file)

In [ ]:
arr=[]
coun=0
for line in np_arr:
    fact=(int(line[0]),int(line[1]),int(line[2]))
    arr.append(get_explanation(fact,relation_to_body,dict_1,dict_2,e1r_e2,re2_e1,set_mapped_train_data))
    coun+=1


In [ ]:
# print(arr)
count=0
for x in arr:
    if x[0]!="":
        count+=1
print(count)

In [ ]:
print(arr[0:100])

In [ ]:
# path=os.path.join(mining_dir,"explanation_test_hits10_not_hits1.pkl")
path=os.path.join(mining_dir,"explanation_test.pkl")
utils.dump_pickle(arr,path)

In [34]:
print(len(np_arr),len(arr))

131 131


In [35]:
lol=utils.load_pickle(path)

In [36]:
print(len(lol))
print(lol[0:100])

131
[((114, 128, 7), (9902, 3719)), ((7, 9, 52), (500, 11085)), ((7, 9, 44), (3674, 361)), ((3, 13, 30), (608, 3435)), ((181, 25, 181), (1684, 1156)), ((32, 30, 32), (4678, 230)), ((64, 53, 3), (6214, 2951)), ((252, 390, 289), (3629, 2507)), ((7, 98, 21), (1337, 12819)), ((30, 32, 84), (230, 648)), ((3, 13, 61), (3887, 12317)), ((43, 91, 195), (101, 914)), ((3, 3, 164), (7717, 6289)), ((23, 28, 132), (4004, 546)), ((4, 5, 171), (10, 3417)), ((2, 184, 54), (973, 6412)), ((1, 225, 210), (2098, 3094)), ((6, 58, 34), (9183, 1313)), (240, -1), ((7, 128, 52), (3089, 4063)), ((16, 27, 26), (975, 10616)), ((11, 10, 72), (250, 21)), ((138, 261, 138), (3872, 2616)), ((9, 7, 9), (2969, 1561)), ((15, 136, 186), (4703, 211)), ((30, 32, 165), (1104, 4797)), ((10, 11, 53), (273, 4098)), ((84, 230, 342), (58, 1278)), ((7, 98, 20), (4300, 1642)), ((7, 98, 50), (420, 4987)), ((181, 25, 219), (3143, 1683)), ((13, 13, 358), (5810, 4177)), ((38, 22, 38), (5029, 5030)), ((9, 52, 35), (9286, 1094)), ((397, 3

In [21]:
# -----------------------------------------------------------
# Mayank
# To do analysis of rules1 vs rules2 vs rules3
relation_to_body_1={}
relation_to_body_1=add_relation_body(rules_1,relation_to_body_1)

relation_to_body_2={}
relation_to_body_2=add_relation_body(rules_2,relation_to_body_2)

relation_to_body_3={}
relation_to_body_3=add_relation_body(rules_3,relation_to_body_3)

In [22]:
arr_1=[]
coun_1=0
arr_2=[]
coun_2=0
arr_3=[]
coun_3=0
for line in np_arr:
    fact=(int(line[0]),int(line[1]),int(line[2]))
    arr_1.append(get_explanation(fact,relation_to_body_1,dict_1,dict_2,e1r_e2,re2_e1,set_mapped_train_data))
    coun_1+=1
    arr_2.append(get_explanation(fact,relation_to_body_2,dict_1,dict_2,e1r_e2,re2_e1,set_mapped_train_data))
    coun_2+=1
    arr_3.append(get_explanation(fact,relation_to_body_3,dict_1,dict_2,e1r_e2,re2_e1,set_mapped_train_data))
    coun_3+=1

NameError: name 'np_arr' is not defined

In [54]:
path=os.path.join(mining_dir,"explanation_test_1.pkl")
utils.dump_pickle(arr_1,path)
path=os.path.join(mining_dir,"explanation_test_2.pkl")
utils.dump_pickle(arr_2,path)
path=os.path.join(mining_dir,"explanation_test_3.pkl")
utils.dump_pickle(arr_3,path)